In [1]:
%reload_ext autoreload
%autoreload 1

# End-to-End example on a Cell Tracking Challenge Dataset

Preparation: 
- Create an environment, install motile and napari: 
```
conda create -n trackathon -c conda-forge -c funkelab -c gurobi ilpy napari
conda activate trackathon
pip install motile
```

- Clone and install the ctctools (as editable)
```
git clone ...
cd ctc-tools
# git switch trackathon
# pip install -e .
# - Create a folder called "data" in working directory
```

In [11]:
import napari
import urllib.request
import zipfile
from tqdm import tqdm
from scipy.spatial.distance import cdist
import itertools
from pathlib import Path
import networkx as nx
import motile
from motile.costs import NodeSelection, EdgeSelection, Appear
from motile.constraints import MaxParents, MaxChildren
from motile.variables import NodeSelected, EdgeSelected
import numpy as np
import pandas as pd

## PART 1: Get some data

We will download the hamster ovaries dataset from the Cell Tracking Challenge. It is a 3D dataset, but small enough to run quickly and cleanly on most laptops.

In [12]:
# Add a utility to make a progress bar when downloading the file
url = "http://data.celltrackingchallenge.net/training-datasets/Fluo-N3DH-CHO.zip"
base_dir = Path("data")
filename = url.split('/')[-1]
ds_name = filename.split('.')[0]
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

In [13]:
if not base_dir.exists():
    base_dir.mkdir(parents=True)
if not (base_dir / filename).exists():
    print(f"Downloading {ds_name} data from the CTC website")
    # Downloading data
    with DownloadProgressBar(unit='B', unit_scale=True,
                                miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, base_dir / filename, reporthook=t.update_to)
    # Unzip the data
    # TODO add a progress bar to zip as well
    with zipfile.ZipFile(base_dir / filename, 'r') as zip_ref:
        zip_ref.extractall(base_dir)

Next we will load the dataset into a set of detections and visualize it in napari. 

In [14]:
# TODO this should go to some core repo
from shared import format_ctc_data, load_tiffs

In [15]:
data_dir = Path("data/Fluo-N3DH-CHO")
seg_dir = data_dir / "01_GT/TRA"
# Load the raw and segmentations
images = load_tiffs(data_dir / "01")
masks = load_tiffs(str(seg_dir))

In [16]:
# Get graph and detections
gt_graph, detections  = format_ctc_data(masks, track_path=seg_dir / "man_track.txt")

- [ ] TODO turn a graph into a track layer

In [17]:
import numpy as np

def to_detections(graph, id_name="track_id"):
    """
    Turn a graph into a track layer.
    """
    view_graph = graph.copy()
    # Get detections from nodes
    nodes = [{"index": k, **v} for k,v in dict(view_graph.nodes(data=True)).items()]
    detections = pd.DataFrame(nodes).set_index("index")
    # Get Division points
    # Since the id of the tracks is going to be specified later, we just keep the resulting edges in a list
    divisions = [id for id, degree in view_graph.out_degree() if degree > 1]
    division_edges = [
            (div, k)
            for div in divisions
            for k in view_graph[div]
    ]
    # Remove division edges from solution graph
    view_graph.remove_edges_from(division_edges)
    # This allows us to get all tracks as connected components
    components = [c for c in nx.weakly_connected_components(view_graph)]
    # We assign an id to each of the connected components
    detections[id_name] = -1 
    for i, c in enumerate(components):
        detections[id_name][list(components[i])] = i
    # Now that we have track ids, we can create a division graph for napari's track layer
    division_graph = {}
    for parent, child in division_edges:
        parent_id = detections.loc[parent][id_name]
        child_id = detections.loc[child][id_name]
        division_graph[int(child_id)] = [int(parent_id)]
    return detections, division_graph

In [18]:
gt_detections, gt_division_graph = to_detections(gt_graph);


/var/folders/dr/zx61t7mx0xl1y9gsljydj6hh0000gq/T/ipykernel_20264/4141931117.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detections[id_name][list(components[i])] = i
/var/folders/dr/zx61t7mx0xl1y9gsljydj6hh0000gq/T/ipykernel_20264/4141931117.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detections[id_name][list(components[i])] = i
/var/folders/dr/zx61t7mx0xl1y9gsljydj6hh0000gq/T/ipykernel_20264/4141931117.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [19]:
id_name = "track_id"
frame_key="t"
location_keys=["x", "y", "z"]
viewer = napari.Viewer()
viewer.add_image(images, name="Raw")
viewer.add_labels(masks, name="Segmentations")
viewer.add_tracks(gt_detections[[id_name, frame_key, *location_keys]].to_numpy(), graph=gt_division_graph, name="Tracks")

<Tracks layer 'Tracks' at 0x2b6c3c510>

## PART 2: Tracking with motile

Now that we have some data, let's try tracking it with motile. 

To begin with, we need to create a candidate graph. We will use the ground truth segmentations that we loaded earlier as detections: these will be the nodes in our graph. 
Between all nodes from timepoint `t` and nodes from timepoint `t + 1`, we add an edge. 

Motile costs are built from attributes (or `features`) given to the nodes and edges of the candidate graph. 
To create these, we add a positive `score` attribute to the nodes to encourage selecting them.
For the edges, we assume that we want nodes that are closer together (in space) to be more likely to be connected. We therefore weight the edges by the euclidian distance between their centroids.


- [ ] TODO : add the Ground truth on the nodes, and create a "GT" element for the edges such that struct SVM can use it
    - Should we use the full ground truth as an input to StructSVM?

In [20]:
# Generate a pseudo-score for the nodes
scored_detections = detections.copy()
scored_detections["score"] = 1
scored_detections.rename(columns={"segmentation_id" : "gt_track_id"}, inplace=True)
# Create nodes frmo this information
cells = scored_detections.reset_index().rename(columns={"index": "id"})[["id", frame_key, *location_keys, "score", "gt_track_id"]].to_dict("records") 

In [21]:
# Generate the edges
edges = [
    # edge for every pair in t and t+1
    # with score the euclidian distance between edges
]
start_t = detections[frame_key].min()
end_t = detections[frame_key].max()

for t in range(start_t, end_t):
    prev_cells = detections[detections[frame_key] == t]
    next_cells = detections[detections[frame_key] == t + 1]
    # This gives pairwise distances as an array where the first index is time t and the second is time t + 1
    distances = cdist(prev_cells[location_keys], next_cells[location_keys])
    # TODO add gt_id=1 if node1 and node2 have same gt_track_id, else 0
    prev = range(len(prev_cells))
    next = range(len(next_cells))
    for i, j in itertools.product(prev, next):
        edges.append({"source": prev_cells.index[i], "target": next_cells.index[j], "distance": distances[i, j]})

Finally, we create a motile TrackGraph from these nodes and edges.

In [22]:
import networkx as nx

In [23]:
# Create a motile candidate graph from detections
track_graph = nx.DiGraph()
track_graph.add_nodes_from([
    (cell['id'], cell)
    for cell in cells
])
track_graph.add_edges_from([
    (edge['source'], edge['target'], edge)
    for edge in edges
])

In [24]:
for u,v in track_graph.edges:
    node_u = track_graph.nodes[u]
    node_v = track_graph.nodes[v]
    selected = 1 if node_u["gt_track_id"] == node_v["gt_track_id"] else 0
    track_graph.edges[(u, v)]["gt"] = selected

## Solve!

- [ ] TODO blah blah blah

In [25]:
candidate_graph = motile.TrackGraph(track_graph)

In [26]:
solver = motile.Solver(candidate_graph)
# Add costs for nodes and edges
# Constant node selection cost, to ensure we choose cells
solver.add_costs(
    NodeSelection(
        weight=-100,
        attribute="score",
    )
)
# Edge cost proportional to the distance between detections
solver.add_costs(
    EdgeSelection(
        weight=1.0,
        attribute='distance'))

# Add a cost for appearance
# In general, we prefer divisions over appearances!
solver.add_costs(Appear(constant=10000.0))

# Add constraints for divisions
solver.add_constraints(MaxParents(1))
solver.add_constraints(MaxChildren(2))

- [ ] Add Struct SVM!

In [27]:
# TODO uncomment if you have much time and many licences
# solver.fit_weights(gt_attribute='gt')

In [28]:
# Solve
solution = solver.solve()

Could not create Gurobi backend: Gurobi error in ./ilpy/impl/solvers/GurobiBackend.cpp:22: PIP license can only be used from gurobipy interface
presolving:
(round 1, fast)       20 del vars, 44 del conss, 0 add conss, 10 chg bounds, 995 chg sides, 995 chg coeffs, 0 upgd conss, 0 impls, 10087 clqs
   (0.1s) running MILP presolver
   (0.1s) MILP presolver found nothing
(round 2, exhaustive) 20 del vars, 44 del conss, 0 add conss, 10 chg bounds, 995 chg sides, 995 chg coeffs, 11773 upgd conss, 0 impls, 10087 clqs
(round 3, medium)     20 del vars, 9036 del conss, 17984 add conss, 10 chg bounds, 995 chg sides, 995 chg coeffs, 11773 upgd conss, 0 impls, 28071 clqs
(round 4, exhaustive) 20 del vars, 9036 del conss, 17984 add conss, 10 chg bounds, 995 chg sides, 1876 chg coeffs, 12668 upgd conss, 0 impls, 28071 clqs
(round 5, medium)     20 del vars, 9931 del conss, 18879 add conss, 10 chg bounds, 995 chg sides, 1876 chg coeffs, 12668 upgd conss, 0 impls, 36746 clqs
(round 6, exhaustive) 20 d

## PART 4: Evaluate our solution

- [ ] TODO run CTC metrics on our solution

First, we need to create a solution graph. We do this by getting a list of all of the nodes and all of the edges that were selected, then reconstructing them into a `TrackGraph`.

In [29]:
node_selected = solver.get_variables(NodeSelected)
edge_selected = solver.get_variables(EdgeSelected)

selected_nodes = []
selected_edges = []
for node, attributes in track_graph.nodes(data=True):
    if solution[node_selected[node]] > 0.5:
        selected_nodes.append((node, attributes))
for u, v, attributes in track_graph.edges(data=True):
    if solution[edge_selected[(u, v)]] > 0.5:
        selected_edges.append((u, v, attributes))

In [30]:
print(f"Selected {100*(len(selected_nodes) / len(cells))}% of all candidate nodes")
print(f"Selected {100*(len(selected_edges) / len(edges))}% of all candidate edges")

Selected 100.0% of all candidate nodes
Selected 9.84381874175099% of all candidate edges


In [31]:
solution_graph = nx.DiGraph()
solution_graph.add_nodes_from(selected_nodes)
solution_graph.add_edges_from(selected_edges)

In [32]:
# TODO run matching
# TODO etc.

from traccuracy import TrackingGraph
from traccuracy import TrackingData

In [33]:
solution_data = TrackingData(TrackingGraph(solution_graph), segmentation=masks)
gt_data = TrackingData(TrackingGraph(track_graph), segmentation=masks)

- [ ] TODO Run evaluation :)

## PART 3: Visualizing our solutions

Now that we have a solution, let's visualize it to see whether it looks sensible.
We're going to add the solution into `napari`. 

The tracks layer in `napari` requires two types of data. 
- an array of detections ordered by `cell_id`, `t`, `x`, `y`, `z`
- a branching graph were keys are nodes, and values are lists of parent nodes. In our case there are only divisions, so the list will be of length one each time.

In [34]:
solution_detections, solution_division_graph = to_detections(solution_graph)
viewer.add_tracks(solution_detections[[id_name, frame_key, *location_keys]].to_numpy(), graph=solution_division_graph, name="Solution")

/var/folders/dr/zx61t7mx0xl1y9gsljydj6hh0000gq/T/ipykernel_20264/4141931117.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detections[id_name][list(components[i])] = i
/var/folders/dr/zx61t7mx0xl1y9gsljydj6hh0000gq/T/ipykernel_20264/4141931117.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detections[id_name][list(components[i])] = i
/var/folders/dr/zx61t7mx0xl1y9gsljydj6hh0000gq/T/ipykernel_20264/4141931117.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

<Tracks layer 'Solution' at 0x2b7ee6550>

## PART 5: Conveniently save everything

- [ ] TODO save the images, candidate graph, ground truth graph, and solution graph into our new data format :)